In [1]:
from adam.core.computations import KinDynComputations
from adam.geometry import utils
import numpy as np
import casadi as cs
import math
import gym_ignition_models
from adam.core.computationsWithHardwareParameters import  KinDynComputationsWithHardwareParameters
from adam.geometry import linkParametric
from urdfpy import xyz_rpy_to_matrix, matrix_to_xyz_rpy

In [2]:
def SX2DM(sx):
    return cs.DM(sx)

# Loading the model 

In [3]:

urdf_path ="/home/carlotta/iit_ws/ergocub-gazebo-simulations/models/stickBot/model.urdf"
urdf_path_old = gym_ignition_models.get_model_file("iCubGazeboV2_5")
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
link_name_list = ['chest']
link_name_list_empty  = []
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
r_hip_yaw_char = linkParametric.JointCharacteristics(-0.055989)
chest_char = linkParametric.LinkCharacteristics(-0.0133,linkParametric.Side.DEPTH, flip_direction=False, calculate_origin_from_dimension = True)
r_shoulder_char = linkParametric.JointCharacteristics(-0.0462, take_half_length=False, flip_direction=False) # rshoulder pitch 
l_shoulder_char = linkParametric.JointCharacteristics(-0.0462, take_half_length=False, flip_direction=False) # l shoulder pitch
neck_fixed_char = linkParametric.JointCharacteristics(-0.0409, take_half_length=False, flip_direction=False) # neck fixed joint
link_name = 'root_link'
link_characteristics = {'chest':chest_char}
joint_characteristics = {'r_shoulder_pitch': r_shoulder_char,'l_shoulder_pitch': l_shoulder_char, 'neck_fixed_joint': neck_fixed_char}
# link_characteristics = {link_name: r_hip_r_char}
# joint_characteristics = {'l_ankle_pitch': r_hip_yaw_char}
# print(joint_characteristics)
root_link = 'root_link'
kinDyn =KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list,link_name_list, root_link, link_characteristics, joint_characteristics)

kinDynNoParametric = KinDynComputations(urdf_path,joints_name_list, root_link)
kynDynParam_NoLink = KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list, link_name_list_empty, root_link)


Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor
Unknown tag: sensor


### Variable definition

In [4]:
# Set joints and base informations
link_original = kinDyn.get_element_by_name(link_name, kinDyn.robot)
r_hip_r_char = linkParametric.LinkCharacteristics(0.058)
width = link_original.visuals[0].geometry.box.size[0]
depth = link_original.visuals[0].geometry.box.size[2]
height = link_original.visuals[0].geometry.box.size[1]

volume = width*depth*height
mass = link_original.inertial.mass
density_num = mass/volume

xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
density =density_num 
lenght_multiplier = 1
gravity =np.array([0, 0, -9.80665, 0, 0, 0])

### Mass and Centroidal Matrix 

In [5]:
[M, J_cmm] = kinDyn.crba()
M = kinDyn.mass_matrix_fun() 
J_cmm = kinDyn.centroidal_momentum_matrix_fun()

# print("M", M(H_b, s, density, lenght_multiplier))
# print("J_cmm",J_cmm(H_b, s, density, lenght_multiplier))

inside the crba link parametric
I <class 'adam.geometry.linkParametric.I_parametric'>
origin [ 0.          0.                 nan  0.          0.         -1.57079633]
Ic [array([[ 2.67135,  0.     ,  0.     ,  0.     ,  0.     , -0.     ],
       [ 0.     ,  2.67135,  0.     , -0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  2.67135,  0.     , -0.     ,  0.     ],
       [ 0.     , -0.     ,  0.     ,  0.01   ,  0.     ,  0.     ],
       [ 0.     ,  0.     , -0.     ,  0.     ,  0.01   ,  0.     ],
       [-0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.01   ]]), array([[ 1.81255   ,  0.        ,  0.        ,  0.        ,  0.13594125,
        -0.        ],
       [ 0.        ,  1.81255   ,  0.        , -0.13594125,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  1.81255   ,  0.        , -0.        ,
         0.        ],
       [ 0.        , -0.13594125,  0.        ,  0.02019559,  0.        ,
         0.        ],
       [ 0.13594125,  0.        ,

In [6]:
M_no_parametric = kinDynNoParametric.mass_matrix_fun()
M_parametric_no_link = kynDynParam_NoLink.mass_matrix_fun()
M_no_parametric_value = M_no_parametric(H_b, s)
# M_no_parametric_value = M_parametric_no_link(H_b,s,density,lenght_multiplier)
M_value = M(H_b, s, density, lenght_multiplier)
# print("M", M_no_parametric(H_b, s,density, lenght_multiplier))



In [7]:
error = 0 
for i in range(29):
    for j in range(29):
        err = float(M_no_parametric_value[i,j] - M_value[i,j])
        print(err)
        error += abs(err)
print('Total Error Mass Matrices', error)

-13.80756489888482
1.6342482922482304e-13
2.930988785010413e-14
nan
nan
nan
nan
nan
8.282263763703668e-14
-2.078198724220215e-15
-1.559863349598345e-14
-6.328271240363392e-15
-3.7192471324942744e-15
-2.3314683517128287e-15
1.6653345369377348e-14
-4.787836793695988e-16
8.271161533457416e-15
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.6342482922482304e-13
-13.807564898884777
5.728750807065808e-14
nan
nan
nan
nan
nan
7.158162951270697e-14
-4.13471340499072e-15
6.952771691715043e-15
-7.674416657721395e-15
3.1780134079895106e-15
-2.042810365310288e-14
-4.996003610813204e-16
-6.147859998861804e-15
-1.0477729794899915e-15
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.375077994860476e-14
4.529709940470639e-14
-13.807564898884962
nan
nan
nan
nan
nan
-9.71445146547012e-16
-3.622102617839573e-15
2.609024107869118e-14
-2.2065682614424986e-15
-1.0234868508263162e-15
2.4702462297909733e-15
1.2101430968414206e-14
9.26342336171615e-16
-3.0808688933348094e-15
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0

In [8]:
J_cmm_no_parameric = kinDynNoParametric.centroidal_momentum_matrix_fun()
J_cmm_no_parameric_value = J_cmm_no_parameric(H_b,s)
J_cmm_parametric_no_link = kynDynParam_NoLink.centroidal_momentum_matrix_fun()
# J_cmm_no_parameric_value = J_cmm_parametric_no_link(H_b,s,density,lenght_multiplier)
J_cmm_value = J_cmm(H_b,s,density, lenght_multiplier)
error = 0 
for i in range(6):
    for j in range (29): 
        err = float(J_cmm_no_parameric_value[i,j] - J_cmm_value[i,j])
        error += abs(err)
print("Total error centroidal momentum matrix", error)



Total error centroidal momentum matrix nan


### Total mass

In [9]:
mass = kinDyn.get_total_mass()
print('mass:\n',mass(density, lenght_multiplier))

mass:
 76.3814


In [10]:
mass_no_param = kinDynNoParametric.get_total_mass()
mass_param_no_link = kynDynParam_NoLink.get_total_mass()
# mass_no_param = mass_param_no_link(density,lenght_multiplier)
mass = kinDyn.get_total_mass()
mass_value = mass(density, lenght_multiplier)
error = float(mass_no_param-mass_value)
print("error mass", error)

error mass -13.80756489888519


### Forward Kinematics

In [11]:
H = kinDyn.forward_kinematics_fun('l_lower_leg')
print('H left sole :\n', H(H_b, s, density, lenght_multiplier))

H left sole :
 @1=0, 
[[-0.304062, -0.839651, -0.450036, 2.32444], 
 [0.922425, -0.377547, 0.0811796, 0.327668], 
 [-0.238072, -0.39044, 0.889313, -1.65326], 
 [@1, @1, @1, 1]]


In [12]:

error_tot = 0 
for item in kinDyn.robot_desc.link_map:
    if(kinDyn.robot_desc.link_map[item].inertial is not None):
        H = kinDyn.forward_kinematics_fun(item)
        H_no_param = kinDynNoParametric.forward_kinematics_fun(item)
        H_link_no_param = kynDynParam_NoLink.forward_kinematics_fun(item)
        H_no_param_value = H_no_param(H_b, s)
        # H_no_param_value = H_link_no_param(H_b,s,density,lenght_multiplier)
        H_value = H(H_b, s, density,lenght_multiplier)
        error = 0 
        for i in range(4):
            for j in range(4):
                err = float(H_value[i,j]- H_no_param_value[i,j])
                error+= abs(err)
        error_tot += error
        # print("error forward kinematic",item, " ", error)
print("total error for the whole kin chain", error_tot)

total error for the whole kin chain 0.10279904047544329


### Center of Mass

In [13]:
com = kinDyn.CoM_position_fun()
print('CoM:\n',com(H_b, s, density, lenght_multiplier))

CoM:
 @1=nan, [@1, @1, @1]


### Jacobian

In [14]:
CoM_no_param = kinDynNoParametric.CoM_position_fun()
CoM_param_no_link = kynDynParam_NoLink.CoM_position_fun()

CoM_no_param_value = CoM_no_param(H_b,s)
# CoM_no_param_value = CoM_param_no_link(H_b,s,density, lenght_multiplier)
CoM_value = com(H_b, s, density, lenght_multiplier)
error = 0
for i in range(3):
    err = float(CoM_no_param_value[i] - CoM_value[i])
    error+= abs(err)
print("error CoM", error)

error CoM nan


In [15]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian left sole\n',J(H_b, s, density, lenght_multiplier))

Jacobian left sole
 @1=1, @2=0, 
[[@1, @2, @2, @2, -0.277569, 0.329293, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.389459, 0.145431, -0.200777, -0.0681683, -0.00966813, 0.053066, @2, @2, @2, @2, @2, @2], 
 [@2, @1, @2, 0.277569, @2, 0.221058, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.243333, -0.277574, -0.105775, 0.214551, 0.0215798, -0.0388424, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @1, -0.329293, -0.221058, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.0316923, 0.323732, -0.0268627, -0.0608682, -7.56591e-05, -0.00221508, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.306886, -0.875154, -0.477491, 0.839651, -0.839651, -0.242659, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, 0.588589, -0.483375, 0.855711, 0.377547, -0.377547, -0.382363, @2, @2, @2, @2, @2, @2], 
 [@2, @2, @2, @2, @2, @1, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, @2, -0.74792, -0.0213072, 0.199401, 0.39044, -0.

In [16]:
error_tot = 0 
for item in kinDyn.robot_desc.link_map:
    if(kinDyn.robot_desc.link_map[item].inertial is not None):
        J_no_param = kinDynNoParametric.jacobian_fun(item)
        J_param_no_link = kynDynParam_NoLink.jacobian_fun(item)
        # J_no_param_value = J_param_no_link(H_b, s,density,lenght_multiplier)
        J_no_param_value = J_no_param(H_b, s)
        J_value = J(H_b,s, density, lenght_multiplier)
        error = 0 
        for i in range(6): 
            for j in range(29): 
                err = float(J_no_param_value[i,j]- J_value[i,j])
                error+ - abs(err)
        error_tot += error
        # print("error Jacobian", error)
print("error whole kin chain jacobian", error)

error whole kin chain jacobian 0


### Relative Jacobian

In [17]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian left sole relative: ', J_r(s,density, lenght_multiplier))

Jacobian left sole relative:  
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.131098, 0.0190342, 0.0599573, -0.13044, -0.0175624, 0.0394536, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.450133, -0.019449, -0.192727, -0.0157253, 0.0374907, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.441256, -0.00419637, 0.219655, -0.0150229, 0.0018562, -0.0369797, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.215295, -0.845117, -0.384443, 0.384443, -0.342901, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -0.501061, 0.327476, -0.327476, 0.817416, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.976549, 0.186319, -0.863112, 0.863112, 0.462871, 0, 0, 0, 0, 0, 0]]


In [18]:
error_tot = 0
for item in kinDyn.robot_desc.link_map:
    if(kinDyn.robot_desc.link_map[item].inertial is not None):
        J_r_no_param = kinDynNoParametric.relative_jacobian_fun('l_sole')
        J_r_param_no_link = kynDynParam_NoLink.relative_jacobian_fun('l_sole')
        J_r_no_param_value = J_r_no_param(s)
        # J_r_no_param_value = J_r_param_no_link(s,density,lenght_multiplier)
        J_value = J_r(s, density, lenght_multiplier)
        error = 0 
        for i in range(6): 
            for j in range(23):
                err = float(J_value[i,j] - J_r_no_param_value[i,j])
                error += abs(err)
        error_tot += error
        # print("error relative jacobian", error)
print("error whole kin chain relative jacobian", error_tot)

error whole kin chain relative jacobian 0.0


In [19]:
tau = kinDyn.rnea()
print('tau: \n', tau(H_b, s, v_b, s_dot, gravity, density, lenght_multiplier))

tau: 
 @1=nan, [@1, @1, @1, @1, @1, @1, @1, @1, @1, 0.314448, 5.45866, 0.034215, -0.961038, 1.73702, 4.24972, 0.326212, 0.00492966, 11.1482, 24.5121, 12.1038, -0.384487, -0.0527662, -2.51121, 0.661282, -33.2711, -12.5542, 8.95489, -4.2468, 2.46324]


In [20]:
tau_no_parametric = kinDynNoParametric.rnea()
tau_parametric_no_link =kynDynParam_NoLink.rnea()
tau_no_parametric_value = tau_no_parametric(H_b, s, v_b, s_dot, gravity)
# tau_no_parametric_value = tau_parametric_no_link(H_b,s, v_b, s_dot, gravity,density,lenght_multiplier)
tau_value = tau(H_b, s, v_b, s_dot, gravity, density,  lenght_multiplier)

error = 0 
for i in range(29):
    err = float (tau_value[i] - tau_no_parametric_value[i])
    error += abs(err)

print("tau error", error)

tau error nan


In [21]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Bias force term:
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.314448, 5.45866, 0.034215, -0.961038, 1.73702, 4.24972, 0.326212, 0.00492966, 11.1482, 24.5121, 12.1038, -0.384487, -0.0527662, -2.51121, 0.661282, -33.2711, -12.5542, 8.95489, -4.2468, 2.46324]


In [22]:
h_no_param = kinDynNoParametric.bias_force_fun()
H_param_no_link= kynDynParam_NoLink.bias_force_fun()
h_no_param_values = H_param_no_link(H_b,s,v_b,s_dot,density,lenght_multiplier)
# h_no_param_values = h_no_param(H_b,s,v_b,s_dot)
h_value = h(H_b,s,v_b,s_dot,density,lenght_multiplier)
error = 0
for i in range(29): 
    err = float(h_value[i]- h_no_param_values[i])
    error += abs(err)
print("bias forces error", error)

bias forces error nan


In [23]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot, density, lenght_multiplier)))

Coriolis force term:
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.537288, -0.293153, 0.398006, -1.14542, 0.976804, -0.571309, 0.141578, 0.101903, 15.653, -9.77833, 15.6698, 3.69891, 0.248293, -2.47651, 1.70896, -6.77696, -0.0454029, 10.6241, -4.24329, 2.43975]


In [24]:
C_no_param = kinDynNoParametric.coriolis_term_fun()
C_param_no_link = kynDynParam_NoLink.coriolis_term_fun()
# C_no_param_value = C_param_no_link(H_b,s,v_b,s_dot,density,lenght_multiplier)
C_no_param_value = C_no_param(H_b,s,v_b, s_dot)
C_value = C(H_b, s,v_b,s_dot,density,lenght_multiplier)
error = 0 
for i in range(29): 
    err = float(C_no_param_value[i] - C_value[i])
    error+= abs(err)
print("Coriolis error", error)

Coriolis error nan


In [25]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s, density, lenght_multiplier)))

Gravity term:
 [-2.84217e-14, 7.81597e-14, 749.046, nan, nan, nan, nan, nan, 1.34559, -0.22284, 5.75181, -0.363791, 0.184381, 0.760214, 4.82103, 0.184633, -0.0969737, -4.50479, 34.2905, -3.56599, -4.0834, -0.301059, -0.0346962, -1.04768, -26.4941, -12.5088, -1.66925, -0.00351557, 0.0234852]


In [26]:
G_no_param = kinDynNoParametric.gravity_term_fun()
G_param_no_link = kynDynParam_NoLink.gravity_term_fun()
G_no_param_value = G_no_param(H_b,s)
# G_no_param_value = G_param_no_link(H_b,s,density,lenght_multiplier)
G_value = G(H_b,s,density,lenght_multiplier)
error = 0 
for i in range(29): 
    err = float(G_no_param_value[i]-G_value[i])
    error += abs(err)
print("gravity term error", error)

gravity term error nan


In [27]:
# frame_1 = 'l_sole'
# frame_2 = 'l_lower_leg'
# H_relative = kinDyn.relative_forward_kinematics_fun(frame_1, frame_2)
# H_frame_one = kinDyn.forward_kinematics_fun(frame_1)
# H_frame_two = kinDyn.forward_kinematics_fun(frame_2)
# H_relative_computed= cs.mtimes(cs.inv(H_frame_one(H_b, s, density, lenght_multiplier)),H_frame_two(H_b, s, density, lenght_multiplier))
# # print("H comptued via relative forward kinematics", H_relative(H_b,s, density, lenght_multiplier))
# print("H computed via forward kin and inverse", H_relative_computed)


In [28]:
mass = kinDyn.get_link_mass(link_name_list[0])
print("mass", mass(density,lenght_multiplier))
print(mass)

mass = kinDyn.get_link_mass('head')
print(mass)

mass 19.5651
mass:(i0,i1)->(o0) SXFunction
1.93676


### Cylinder Link 

In [29]:
link_original = kinDyn.get_element_by_name('l_lower_leg', kinDyn.robot)
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
print(link_original.visuals[0].geometry.cylinder)
length = link_original.visuals[0].geometry.cylinder.length 
radius = link_original.visuals[0].geometry.cylinder.radius
volume = math.pi*radius**2*length
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('radius=', radius)
print('length=', length)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])
link_parametric =linkParametric.linkParametric('l_lower_leg', 1.0,density_num, kinDyn.robot, link_original, r_hip_r_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new[1])
print('length=', link_parametric.visual_data_new[0])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


****************ORIGINAL LINK******************
radius= 0.065
length= 0.2
origin= [-0.02       -0.015      -0.13        0.          0.         -1.57079633]
volume= 0.002654645792283376
mass= 5.66584
density= 2134.3111071426842
inertia xx= 0.02487067683333334
inertia yy= 0.02487067683333334
inertia zz= 0.011969087000000002
****************PARAMETRIC LINK****************
radius= 0.065
length= 0.2
origin= @1=0, [-0.02, -0.015, -0.13, @1, @1, -1.5708]
volume= 0.00265465
mass= 5.66584
density= 2134.3111071426842
inertia xx= 0.0248707
inertia yy= 0.0248707
inertia zz= 0.0119691


### Cylinder Link Parametric

In [30]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_parametric =linkParametric.linkParametric('l_lower_leg', l_m,rho, kinDyn.robot, link_original, r_hip_r_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new[1])
print('length=', link_parametric.visual_data_new[0])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

****************PARAMETRIC LINK****************
radius= 0.065
length= (0.2*l_m)
origin= @1=0, [-0.02, -0.015, (-0.03-((0.2*l_m)/2)), @1, @1, -1.5708]
volume= (0.0132732*(0.2*l_m))
mass= ((0.0132732*(0.2*l_m))*rho)
density= rho
inertia xx= @1=(0.2*l_m), (((0.0132732*@1)*rho)*((0.012675+sq(@1))/12))
inertia yy= @1=(0.2*l_m), (((0.0132732*@1)*rho)*((0.012675+sq(@1))/12))
inertia zz= ((0.004225*((0.0132732*(0.2*l_m))*rho))/2)


### Box Link 

In [31]:
link_original = kinDyn.get_element_by_name('chest', kinDyn.robot)
chest_char = linkParametric.LinkCharacteristics(0.0, linkParametric.Side.DEPTH, False, False)

width = link_original.visuals[0].geometry.box.size[0]
depth = link_original.visuals[0].geometry.box.size[2]
height = link_original.visuals[0].geometry.box.size[1]

volume = width*depth*height
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('width=', width)
print('depth=', depth)
print('height=', height)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])

link_parametric =linkParametric.linkParametric('chest', 1.0,density_num, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('width=', link_parametric.visual_data_new[0])
print('depth=', link_parametric.visual_data_new[2])
print('height=', link_parametric.visual_data_new[1])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


****************ORIGINAL LINK******************
width= 0.2041
depth= 0.2092
height= 0.2152
origin= [ 0.          0.          0.091388    0.          0.         -1.57079633]
volume= 0.009188549344000001
mass= 5.75757
density= 626.6027187152905
inertia xx= 0.0432180860748
inertia yy= 0.04098499815637501
inertia zz= 0.042206754510375005
****************PARAMETRIC LINK****************
width= 0.2041
depth= 0.2092
height= 0.2152
origin= [ 0.          0.          0.091388    0.          0.         -1.57079633]
volume= 0.00918855
mass= 5.75757
density= 626.6027187152905
inertia xx= 0.0432181
inertia yy= 0.040985
inertia zz= 0.0422068


### Box Link Parametric

In [32]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_parametric =linkParametric.linkParametric('chest', l_m,rho, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('width=', link_parametric.visual_data_new[0])
print('depth=', link_parametric.visual_data_new[2])
print('height=', link_parametric.visual_data_new[1])
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

****************PARAMETRIC LINK****************
width= 0.2041
depth= (0.2092*l_m)
height= 0.2152
origin= [ 0.          0.          0.091388    0.          0.         -1.57079633]
volume= (0.0439223*(0.2092*l_m))
mass= ((0.0439223*(0.2092*l_m))*rho)
density= rho
inertia xx= @1=(0.2092*l_m), ((((0.0439223*@1)*rho)*(0.046311+sq(@1)))/12)
inertia yy= @1=(0.2092*l_m), ((((0.0439223*@1)*rho)*(0.0416568+sq(@1)))/12)
inertia zz= ((0.0879679*((0.0439223*(0.2092*l_m))*rho))/12)


### Sphere Link

In [33]:
link_original = kinDyn.get_element_by_name('l_elbow_1', kinDyn.robot)

radius = link_original.visuals[0].geometry.sphere.radius

volume = 4*(math.pi*radius**3)/3
mass = link_original.inertial.mass
density_num = mass/volume
print('****************ORIGINAL LINK******************')
print('radius=', radius)
print('origin=',matrix_to_xyz_rpy(link_original.inertial.origin))
print('volume=', volume)
print('mass=', mass)
print('density=', density_num)
print('inertia xx=', link_original.inertial.inertia[0,0])
print('inertia yy=', link_original.inertial.inertia[1,1])
print('inertia zz=', link_original.inertial.inertia[2,2])

link_parametric =linkParametric.linkParametric('l_elbow_1', 1.0,density_num, kinDyn.robot, link_original, chest_char)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new)
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)

****************ORIGINAL LINK******************
radius= 0.04
origin= [-0.02        0.         -0.02       -0.07156449 -0.06196999 -1.29535745]
volume= 0.00026808257310632905
mass= 0.230254
density= 858.892084375342
inertia xx= 0.01
inertia yy= 0.01
inertia zz= 0.01
****************PARAMETRIC LINK****************
radius= 0.04
origin= [-0.02        0.         -0.02       -0.07156449 -0.06196999 -1.29535745]
volume= 0.00026808257310632905
mass= 0.230254
density= 858.892084375342
inertia xx= 0.00014736256
inertia yy= 0.00014736256
inertia zz= 0.00014736256


### Sphere Link Parametric 

In [34]:
rho = cs.SX.sym('rho', 1)
l_m = cs.SX.sym('l_m',1)
link_original = kinDyn.get_element_by_name('l_elbow_1', kinDyn.robot)
link_parametric =linkParametric.linkParametric('l_elbow_1', l_m,rho, kinDyn.robot, link_original)
print('****************PARAMETRIC LINK****************')
print('radius=', link_parametric.visual_data_new)
print('origin=', link_parametric.origin)
print('volume=', link_parametric.volume)
print('mass=', link_parametric.mass)
print('density=', link_parametric.density)
print('inertia xx=', link_parametric.I.ixx)
print('inertia yy=', link_parametric.I.iyy)
print('inertia zz=', link_parametric.I.izz)


****************PARAMETRIC LINK****************
radius= (0.04*l_m)
origin= [-0.02        0.         -0.02       -0.07156449 -0.06196999 -1.29535745]
volume= @1=(0.04*l_m), ((12.5664*(@1*sq(@1)))/3)
mass= @1=(0.04*l_m), (((12.5664*(@1*sq(@1)))/3)*rho)
density= rho
inertia xx= @1=(0.04*l_m), (((2*(((12.5664*(@1*sq(@1)))/3)*rho))*sq(@1))/5)
inertia yy= @1=(0.04*l_m), (((2*(((12.5664*(@1*sq(@1)))/3)*rho))*sq(@1))/5)
inertia zz= @1=(0.04*l_m), (((2*(((12.5664*(@1*sq(@1)))/3)*rho))*sq(@1))/5)


In [35]:
r_hip_r_char = linkParametric.LinkCharacteristics(-0.03)
r_hip_yaw_char = linkParametric.JointCharacteristics(-0.055989)
joint_original = kinDyn.get_joint_by_name('l_ankle_pitch',kinDyn.robot)
link_original = kinDyn.get_element_by_name('l_lower_leg', kinDyn.robot)
link_parametric =linkParametric.linkParametric('l_lower_leg', l_m,rho, kinDyn.robot, link_original, r_hip_r_char)
joint_parametric = linkParametric.jointParametric('l_ankle_pitch', link_parametric, joint_original, r_hip_yaw_char)
print(matrix_to_xyz_rpy(joint_original.origin))
print(joint_parametric.origin)

[ 0.       -0.0005   -0.255989  0.        0.        0.      ]
@1=0, [@1, -0.0005, (-0.055989-(0.2*l_m)), @1, @1, @1]
